In [1]:
# Import system libs
import os
import time
import shutil
import pathlib
import itertools
from PIL import Image

# Import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Import deep learning libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Define data directory
data_dir = "F:\\Work\\Traditional Food Classification"
filepaths = []
labels = []

folds = os.listdir(data_dir)
for fold in folds:
    file_path = os.path.join(data_dir, fold)
    if not os.path.isdir(file_path):
        continue
    fpath = os.listdir(file_path)
    
    for f in fpath:
        fil_path = os.path.join(file_path, f)
        filepaths.append(fil_path)
        labels.append(fold)

f_series = pd.Series(filepaths, name='filepaths')
l_series = pd.Series(labels, name='labels')
df = pd.concat([f_series, l_series], axis=1)

# Train dataframe 80% train 20% dummy
train_df, dummy_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=43)

# Valid and test dataframe 10% validate 10% test
valid_df, test_df = train_test_split(dummy_df, train_size=0.5, shuffle=True, random_state=43)

# Create image generator with data augmentation for training
batch_size = 16
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

# Define data augmentation for training
tr_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,       # Increase rotation range
    width_shift_range=0.3,   # Increase horizontal shift
    height_shift_range=0.3,  # Increase vertical shift
    shear_range=0.3,         # Increase shear transformation
    zoom_range=0.3,          # Increase zoom
    horizontal_flip=True,    # Random horizontal flip
    brightness_range=[0.7, 1.3],  # Adjust brightness
    fill_mode='nearest'      # Filling in pixels after transformations
)
# No augmentation for validation and testing
ts_gen = ImageDataGenerator(rescale=1./255)

# Apply the generators
train_gen = tr_gen.flow_from_dataframe(
    train_df, x_col='filepaths', y_col='labels', target_size=img_size, class_mode='categorical',
    color_mode='rgb', shuffle=True, batch_size=batch_size)

valid_gen = ts_gen.flow_from_dataframe(
    valid_df, x_col='filepaths', y_col='labels', target_size=img_size, class_mode='categorical',
    color_mode='rgb', shuffle=True, batch_size=batch_size)

test_gen = ts_gen.flow_from_dataframe(
    test_df, x_col='filepaths', y_col='labels', target_size=img_size, class_mode='categorical',
    color_mode='rgb', shuffle=False, batch_size=batch_size)

# Define input image dimensions
class_count = len(list(train_gen.class_indices.keys()))  # Define number of classes

# Define the input layer
inputs = tf.keras.Input(shape=img_shape)

# Add Gaussian noise layer
x = GaussianNoise(0.1)(inputs)

# Load the pre-trained VGG16 model without the top layer
base_model = tf.keras.applications.VGG16(include_top=False, weights="imagenet", input_tensor=x)

# Unfreeze the last few layers for fine-tuning
for layer in base_model.layers[-4:]:  # Unfreeze last 4 layers
    layer.trainable = True

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
x = Dense(512, kernel_regularizer=regularizers.l2(0.01), activation='relu')(x)
x = Dropout(rate=0.5)(x)
outputs = Dense(class_count, activation='softmax')(x)

# Create the final model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    tf.keras.callbacks.ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)  # Reduce learning rate on plateau
]

# Train the model
history = model.fit(
    train_gen, 
    epochs=25,  # Increase epochs for better learning
    verbose=1, 
    validation_data=valid_gen, 
    shuffle=False,
    callbacks=callbacks
)

# Evaluate the model
loss, accuracy = model.evaluate(test_gen, verbose=1)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Found 872 validated image filenames belonging to 7 classes.
Found 109 validated image filenames belonging to 7 classes.
Found 108 validated image filenames belonging to 7 classes.
Epoch 1/25
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.2735 - loss: 6.9739  
Epoch 1: val_loss improved from inf to 7.90695, saving model to best_model.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 343s 6s/step - accuracy: 0.2745 - loss: 6.9697 - val_accuracy: 0.2385 - val_loss: 7.9070 - learning_rate: 1.0000e-04
Epoch 2/25
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 23s/step - accuracy: 0.3721 - loss: 6.2195     
Epoch 2: val_loss improved from 7.90695 to 7.31646, saving model to best_model.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1242s 23s/step - accuracy: 0.3737 - loss: 6.2155 - val_accuracy: 0.3578 - val_loss: 7.3165 - learning_rate: 1.0000e-04
Epoch 3/25
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.5931 - loss: 5.4889  
Epoch 3: val_loss improved from 7.31646 to 5.27511, saving model to best_model.keras
55/55 ━━━━━━━━━━━━━━

In [2]:
# Make predictions on the test data
test_gen.reset()  # Ensure the generator is reset to avoid data shuffling issues
predictions = model.predict(test_gen, verbose=1)
y_pred = np.argmax(predictions, axis=1)
y_true = test_gen.classes  # True labels

# Get class indices and map them to class names
class_indices = {v: k for k, v in test_gen.class_indices.items()}
y_true_labels = [class_indices[label] for label in y_true]
y_pred_labels = [class_indices[label] for label in y_pred]

# Classification report
report = classification_report(y_true, y_pred, target_names=list(test_gen.class_indices.keys()), digits=4)
print("Classification Report:\n", report)

# Compute metrics
conf_matrix = confusion_matrix(y_true, y_pred)
tp = np.diag(conf_matrix)
sensitivity = tp / np.sum(conf_matrix, axis=1)
sensitivity = np.nan_to_num(sensitivity)  # Handle any NaNs

7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step 
Classification Report:
               precision    recall  f1-score   support

     Biriani     0.7600    0.9500    0.8444        20
   Dim bhuna     1.0000    0.7692    0.8696        13
 Panta Ilish     1.0000    0.3750    0.5455         8
  Roshogolla     0.7500    1.0000    0.8571        18
      fuchka     0.8889    0.6667    0.7619        24
   kala vuna     0.6154    0.8889    0.7273         9
    khichuri     0.8667    0.8125    0.8387        16

    accuracy                         0.8056       108
   macro avg     0.8401    0.7803    0.7778       108
weighted avg     0.8374    0.8056    0.7985       108



In [ ]:
#furthure check

In [2]:
# Import system libs
import os
import time
import shutil
import pathlib
import itertools
from PIL import Image

# Import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Import deep learning libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Define data directory
data_dir = "F:\\Work\\Traditional Food Classification"
filepaths = []
labels = []

folds = os.listdir(data_dir)
for fold in folds:
    file_path = os.path.join(data_dir, fold)
    if not os.path.isdir(file_path):
        continue
    fpath = os.listdir(file_path)
    
    for f in fpath:
        fil_path = os.path.join(file_path, f)
        filepaths.append(fil_path)
        labels.append(fold)

f_series = pd.Series(filepaths, name='filepaths')
l_series = pd.Series(labels, name='labels')
df = pd.concat([f_series, l_series], axis=1)

# Train dataframe 80% train 20% dummy
train_df, dummy_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=43)

# Valid and test dataframe 10% validate 10% test
valid_df, test_df = train_test_split(dummy_df, train_size=0.5, shuffle=True, random_state=43)

# Create image generator with data augmentation for training
batch_size = 16
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

# Define data augmentation for training
tr_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,       # Increase rotation range
    width_shift_range=0.3,   # Increase horizontal shift
    height_shift_range=0.3,  # Increase vertical shift
    shear_range=0.3,         # Increase shear transformation
    zoom_range=0.3,          # Increase zoom
    horizontal_flip=True,    # Random horizontal flip
    brightness_range=[0.7, 1.3],  # Adjust brightness
    fill_mode='nearest'      # Filling in pixels after transformations
)
# No augmentation for validation and testing
ts_gen = ImageDataGenerator(rescale=1./255)

# Apply the generators
train_gen = tr_gen.flow_from_dataframe(
    train_df, x_col='filepaths', y_col='labels', target_size=img_size, class_mode='categorical',
    color_mode='rgb', shuffle=True, batch_size=batch_size)

valid_gen = ts_gen.flow_from_dataframe(
    valid_df, x_col='filepaths', y_col='labels', target_size=img_size, class_mode='categorical',
    color_mode='rgb', shuffle=True, batch_size=batch_size)

test_gen = ts_gen.flow_from_dataframe(
    test_df, x_col='filepaths', y_col='labels', target_size=img_size, class_mode='categorical',
    color_mode='rgb', shuffle=False, batch_size=batch_size)

# Define input image dimensions
class_count = len(list(train_gen.class_indices.keys()))  # Define number of classes

# Define the input layer
inputs = tf.keras.Input(shape=img_shape)

# Add Gaussian noise layer
x = GaussianNoise(0.1)(inputs)

# Load the pre-trained VGG16 model without the top layer
base_model = tf.keras.applications.VGG16(include_top=False, weights="imagenet", input_tensor=x)

# Unfreeze the last few layers for fine-tuning
for layer in base_model.layers[-4:]:  # Unfreeze last 4 layers
    layer.trainable = True

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
x = Dense(512, kernel_regularizer=regularizers.l2(0.01), activation='relu')(x)
x = Dropout(rate=0.5)(x)
outputs = Dense(class_count, activation='softmax')(x)

# Create the final model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    tf.keras.callbacks.ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)  # Reduce learning rate on plateau
]

# Train the model

# Evaluate the model


Found 872 validated image filenames belonging to 7 classes.
Found 109 validated image filenames belonging to 7 classes.
Found 108 validated image filenames belonging to 7 classes.


In [3]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gaussian_noise (GaussianNoise)       │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │              

 Total params: 14,982,983 (57.16 MB)

 Trainable params: 14,981,959 (57.15 MB)

 Non-trainable params: 1,024 (4.00 KB)